# <span style="caret-color: rgb(0, 0, 0); color: rgb(0, 0, 0); font-family: -webkit-standard; font-size: medium;">MySQL Database Model for Ruminant Livestock Analysis</span>

### 1. **Cows Table**:

This table stores basic information about each cow, such as breed, age, and lactation stage

In [ ]:
CREATE TABLE cows (
    cow_id INT PRIMARY KEY AUTO_INCREMENT,  -- Unique cow identifier
    cow_name VARCHAR(50),                   -- Optional name for the cow
    breed VARCHAR(50),                      -- Breed of the cow
    birth_date DATE,                        -- Birthdate of the cow
    lactation_stage INT,                    -- Lactation stage (e.g., 1st, 2nd, 3rd stage)
    age INT AS (TIMESTAMPDIFF(YEAR, birth_date, CURDATE())),  -- Calculated age from birth date
    health_status VARCHAR(100)              -- General health status (healthy, under observation, etc.)
);


### 2. **Feed Intake Table**:

This table tracks the daily feed intake of each cow, including the amount of feed consumed and its nutritional content.

In [ ]:
CREATE TABLE feed_intake (
    intake_id INT PRIMARY KEY AUTO_INCREMENT,   -- Unique feed intake record ID
    cow_id INT,                                 -- Reference to cow ID from cows table
    intake_date DATE,                           -- Date of feed intake
    feed_type VARCHAR(50),                      -- Type of feed (e.g., hay, silage, concentrate)
    amount_kg FLOAT,                            -- Amount of feed consumed in kilograms
    protein_percent FLOAT,                      -- Protein percentage in feed
    fat_percent FLOAT,                          -- Fat percentage in feed
    energy_mj FLOAT,                            -- Energy content of the feed in megajoules
    FOREIGN KEY (cow_id) REFERENCES cows(cow_id)  -- Establish relationship with cows table
);


### 3. **Milk Yield Table**:

This table records daily milk yield for each cow, along with fat and protein content of the milk.

In [ ]:
CREATE TABLE milk_yield (
    yield_id INT PRIMARY KEY AUTO_INCREMENT,  -- Unique milk yield record ID
    cow_id INT,                               -- Reference to cow ID from cows table
    yield_date DATE,                          -- Date of milk yield recording
    milk_yield_liters FLOAT,                  -- Amount of milk produced in liters
    fat_content_percent FLOAT,                -- Percentage of fat in the milk
    protein_content_percent FLOAT,            -- Percentage of protein in the milk
    FOREIGN KEY (cow_id) REFERENCES cows(cow_id)  -- Establish relationship with cows table
);


### 4. **Health Records Table**:

This table keeps track of the cow's health status, any diseases, and treatments administered.

In [ ]:
CREATE TABLE health_records (
    health_id INT PRIMARY KEY AUTO_INCREMENT,  -- Unique health record ID
    cow_id INT,                                -- Reference to cow ID from cows table
    record_date DATE,                          -- Date of health record
    temperature FLOAT,                         -- Cow's body temperature
    heart_rate INT,                            -- Cow's heart rate
    disease VARCHAR(100),                      -- Disease diagnosed (if any)
    treatment VARCHAR(100),                    -- Treatment given (if any)
    FOREIGN KEY (cow_id) REFERENCES cows(cow_id)  -- Establish relationship with cows table
);


### 5. **Feed Costs Table** :

If you want to perform a feed cost optimization analysis, this table stores the cost of each feed type

In [ ]:
CREATE TABLE feed_costs (
    feed_type VARCHAR(50) PRIMARY KEY,  -- Type of feed (same as feed_intake feed_type)
    cost_per_kg DECIMAL(5, 2)           -- Cost of the feed per kilogram
);


### 6. **Sample Queries for Analysis**

Now that the tables are set up, you can use these sample SQL queries to analyze the data.

#### a. **Total Milk Yield per Cow**:

This query calculates the total milk yield for each cow over a specific time period.

In [ ]:
SELECT 
    cow_id, 
    SUM(milk_yield_liters) AS total_milk_yield 
FROM 
    milk_yield
WHERE 
    yield_date BETWEEN '2024-01-01' AND '2024-12-31'
GROUP BY 
    cow_id;


#### b. **Average Feed Intake per Cow**:

This query calculates the average daily feed intake for each cow, including the breakdown of protein, fat, and energy content

In [ ]:
SELECT 
    cow_id,
    AVG(amount_kg) AS avg_feed_amount_kg,
    AVG(protein_percent) AS avg_protein_percent,
    AVG(fat_percent) AS avg_fat_percent,
    AVG(energy_mj) AS avg_energy_mj
FROM 
    feed_intake
WHERE 
    intake_date BETWEEN '2024-01-01' AND '2024-12-31'
GROUP BY 
    cow_id;


#### c. **Milk Yield Correlation with Feed Intake**:

This query joins the feed intake and milk yield data to analyze the relationship between feed intake and milk yield.

In [ ]:
SELECT 
    fi.cow_id, 
    fi.intake_date,
    fi.amount_kg, 
    fi.protein_percent, 
    fi.fat_percent, 
    fi.energy_mj, 
    my.milk_yield_liters 
FROM 
    feed_intake fi
JOIN 
    milk_yield my 
    ON fi.cow_id = my.cow_id AND fi.intake_date = my.yield_date;


#### d. **Health Status Report**:

This query retrieves all cows with abnormal health records, including high temperature or low milk yield.

In [ ]:
SELECT 
    h.cow_id, 
    h.record_date, 
    h.temperature, 
    h.disease, 
    m.milk_yield_liters
FROM 
    health_records h
JOIN 
    milk_yield m 
    ON h.cow_id = m.cow_id AND h.record_date = m.yield_date
WHERE 
    h.temperature > 39.0  -- Abnormally high temperature
    OR m.milk_yield_liters < 10;  -- Low milk yield threshold
